In [1]:
import torch
import time
import pandas as pd
import numpy as np
import re
import gensim
import collections
import nltk
from nltk import word_tokenize
from stopwordsallforms import STOPWORDS as arb_stopwords
import elements
#nltk.download("stopwords")
nltk.download('punkt')
import pandas as pd

/home/amr/anaconda3/envs/torch/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to /home/amr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data
### Helping Functions

In [2]:
def normalization(t):
    return t.translate(t.maketrans(''.join(elements.ALEF_HAMZA_FORMS+elements.NON_ALIF_HAMZA_FORMS+tuple(elements.ALIF_MAQSURA)),"اااويي"))

In [3]:
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

In [4]:
arb_stopwords = [clean_str(normalization(i)) for  i in set(arb_stopwords.keys())]

In [5]:
def stopwords_removal(t,stop=arb_stopwords):
    return ' '.join([word for word in word_tokenize(t) if word not in stop])

### Data

In [7]:
data= np.load('../translation project/AD_NMT-master/LAV-MSA-2-both.pkl',allow_pickle=True)

In [8]:
data[0] # lav , msa

['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش',
 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة']

In [9]:
# extract only msa text
msa=[]
for i,ex in enumerate(data):
    msa_text = stopwords_removal(clean_str(normalization(ex[1])))
    data[i][1] = msa_text
    msa.append(msa_text)

In [10]:
msa = ' '.join(msa)

Dictionaries

In [12]:
msa_d=collections.Counter(msa.split())

In [13]:
min_count = 2

In [15]:
idx2msa = np.array([word for word,freq in msa_d.items() if freq > min_count ])

In [16]:
msa2idx = {word:i for i,word in enumerate(idx2msa)}

In [18]:
msa_data = [' '.join([i for i in t[1].split() if (msa2idx.get(i,-1) != -1)]) for t in data]

Load Embeddings

In [28]:
t_model = gensim.models.Word2Vec.load('../resources/models/word vectors/word2vec/wiki/full_grams_cbow_100_wiki/full_grams_cbow_100_wiki.mdl')

In [48]:
list(elements.BASIC_LETTERS)
i2v = {}
for letter in i2l:
    i2v[i] = t_model.wv.get_vector(letter)
i2l +=  ' '
l2i = {v:i for i,v in enumerate(i2l)}

Deep Learning

In [49]:
from torch.utils.data import Dataset,DataLoader

In [64]:
np.random.choice(,2,replace=False)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [122]:
np.random.randint(0,5,1)

array([4])

In [123]:
def noise(txt):
    noise_sz = np.random.randint(0,len(txt),1)
    replace_idx = np.random.choice(len(txt),noise_sz,replace=False)
    letters_idx = np.random.choice(len(i2l),noise_sz,replace=False)
    txt = list(txt)
    for rep,let in zip(replace_idx,letters_idx):
        txt[rep] = i2l[let]
    return ''.join(txt)

In [131]:
class arrDs(Dataset):
    def __init__(self,txt_list,l2i):
        self.data = txt_list
        self.l2i = l2i
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        X = noise(self.data[idx])
        Y = self.data[idx]
        
        X = torch.tensor([self.l2i[i] for i in X])
        Y = torch.tensor([self.l2i[i] for i in Y])
        #numerilize
        return torch.stack([X,Y])

In [132]:
ds = arrDs(msa_data,l2i)

In [135]:
dl = DataLoader(ds,batch_size=2)

In [137]:
from torch.nn.utils.rnn import pad_sequence 

In [138]:
?pad_sequence

In [136]:
for i in dl:
    print(i)

RuntimeError: stack expects each tensor to be equal size, but got [2, 31] at entry 0 and [2, 15] at entry 1